###### using http://www.easy-tensorflow.com/autoencoders/noise-removal

In [1]:
import numpy as np
import scipy.sparse as ss
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle

/home/esadrfa/libs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# data = ss.random(1000, 23000000, density=0.003, 
#                  format='csr', dtype=np.float32, random_state=10)
data = ss.random(4, 5, density=0.7, 
                 format='csr', dtype=np.float32, random_state=10)

In [3]:
# hyper-parameters
logs_path = "./logs/noiseRemoval_test_csr_matrix"  # path to the folder that we want to save the logs for Tensorboard
learning_rate = 0.001  # The optimization learning rate
epochs = 10  # Total number of training epochs
batch_size = 100  # Training batch size
display_freq = 100  # Frequency of displaying the training results

# Network Parameters
num_features = data.shape[1]

# number of units in the hidden layer
# h1 = 1000
h1 = 2

In [9]:
coo_matrix = data.T.tocoo()
tf_coo_matrix = tf.SparseTensorValue(
    indices=np.array([coo_matrix.row, coo_matrix.col], dtype=np.int).T,
    values=coo_matrix.data,
    dense_shape=coo_matrix.shape)

X = tf.SparseTensor.from_value(tf_coo_matrix)

In [12]:
X.get_shape()

TensorShape([Dimension(5), Dimension(4)])

In [16]:
X.values

<tf.Tensor 'SparseTensor_3/values:0' shape=(14,) dtype=float32>

In [14]:
x_original = tf.sparse_placeholder(tf.float32\
                           , shape=[batch_size, num_features]\
                           , name='X_original')

# W = weight_variable(name, shape=[in_dim, num_units])
initer = tf.truncated_normal_initializer(stddev=0.01)
W = tf.get_variable('Weights_test', dtype=tf.float32, 
                    shape=[h1, num_features], initializer=initer)

# b = bias_variable(name, [num_units])
initial = tf.constant(0., shape=[h1,1], dtype=tf.float32)
b = tf.get_variable('bias_test', dtype=tf.float32, initializer=initial)

#         layer = tf.matmul(x, W)    #replaced with a sparse version

### Dimensions
##### W:    n_h x n_f ==> 2 x 5 (num_hidden x num_features)
##### X:     n_f x n_d ==> 5 x 4 (num_features x num_data_points)
##### b:     n_h x 1

In [15]:
layer = tf.sparse_tensor_dense_matmul(trainData, W)
layer += b

ValueError: Tensor("Weights_test:0", shape=(2, 5), dtype=float32_ref) must be from the same graph as Tensor("SparseTensor_2/indices:0", shape=(14, 2), dtype=int64).

In [ ]:

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

out_values = fc_layer(trainData, num_features, h1, 'TestMatMult', use_relu=False)


In [ ]:
# weight and bais wrappers
def weight_variable(name, shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W_' + name,
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(name, shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b_' + name,
                           dtype=tf.float32,
                           initializer=initial)

def fc_layer(x, input_dim, num_units, name, use_relu=True):
    """
    Create a fully-connected layer
    :param x: input from previous layer
    :param num_units: number of hidden units in the fully-connected layer
    :param name: layer name
    :param use_relu: boolean to add ReLU non-linearity (or not)
    :return: The output array
    """
    with tf.variable_scope(name):
#         in_dim = x.get_shape()[1] 
        in_dim = input_dim
        W = weight_variable(name, shape=[in_dim, num_units])
        tf.summary.histogram('W', W)
        b = bias_variable(name, [num_units])
        tf.summary.histogram('b', b)
#         layer = tf.matmul(x, W)    #replaced with a sparse version
        layer = tf.sparse_tensor_dense_matmul(x, W)
        layer += b
        if use_relu:
            layer = tf.nn.relu(layer)
#         return tf.contrib.layers.dense_to_sparse(layer)
    zero = tf.constant(0, dtype=tf.float32)
    where = tf.not_equal(layer, zero)
    indices = tf.where(where)
    values = tf.gather_nd(layer, indices)
    return tf.SparseTensor(indices, values, layer.shape)


##### The below code allow to reset the graph and modify the layers, in case they  have already configured

In [13]:
tf.reset_default_graph()

##### The error for not setting the first dimension on the sparse_placeholder
<pre><code>
Code:
x_original = tf.sparse_placeholder(tf.float32, shape=[None, num_features]\
                           , name='X_original')
Error:
ValueError: Cannot convert a partially known TensorShape to a Tensor: (?, 5)
</code></pre>

In [ ]:
# # sparse version
# # Create graph
# # Placeholders for inputs (x), outputs(y)
# with tf.variable_scope('Input'):
#     x_original = tf.sparse_placeholder(tf.float32\
#                            , shape=[batch_size, num_features]\
#                            , name='X_original')
#     out_values = tf.sparse_placeholder(tf.float32\
#                            , shape=[batch_size, num_features]\
#                            , name='out_values')

# fc1 = fc_layer(x_original, num_features, h1, 'Hidden_layer', use_relu=True)
# out = fc_layer(fc1, h1, num_features, 'Output_layer', use_relu=False)

In [ ]:
# dense version
# # Create graph
# # Placeholders for inputs (x), outputs(y)
# with tf.variable_scope('Input'):
#     x_original = tf.placeholder(tf.float32\
#                            , shape=[batch_size, num_features]\
#                            , name='X_original')
#     out_values = tf.placeholder(tf.float32\
#                            , shape=[batch_size, num_features]\
#                            , name='out_values')

# fc1 = fc_layer(x_original, num_features, h1, 'Hidden_layer', use_relu=True)
# out = fc_layer(fc1, h1, num_features, 'Output_layer', use_relu=False)

In [ ]:
# sparse version
# Create graph
# Placeholders for inputs (x), outputs(y)
with tf.variable_scope('Input'):
    x_original = tf.sparse_placeholder(tf.float32\
                           , shape=[batch_size, num_features]\
                           , name='X_original')
#     out_values = tf.sparse_placeholder(tf.float32\
#                            , shape=[batch_size, num_features]\
#                            , name='out_values')

    

fc1 = fc_layer(x_original, num_features, h1, 'Hidden_layer', use_relu=True)
# out = fc_layer(fc1, h1, num_features, 'Output_layer', use_relu=False)

###### make_negative sparseTensor

https://stackoverflow.com/a/40666375/2674061
<pre><code>
out.get_shape().as_list()[0]
neg_eye = tf.scalar_mul(-1, tf.eye(out.get_shape().as_list()[1]))
negative_val = tf.sparse_tensor_dense_matmul(out, neg_eye)     
diff_sparsetensor = tf.sparse_add(x_original , negative_val)
square_diff = tf.square(diff_sparsetensor)
</code></pre>

In [ ]:
# # Define the loss function, optimizer, and accuracy for sparseTensors
# with tf.variable_scope('Train'):
#     with tf.variable_scope('Loss'):
# #         loss = tf.sparse_reduce_sum_sparse(tf.losses.sparse_softmax_cross_entropy(x_original, out), name='loss')
#         neg_eye = tf.scalar_mul(-1, tf.eye(out.get_shape().as_list()[1]))
#         negative_val = tf.sparse_tensor_dense_matmul(out, neg_eye)     
#         square_diff = tf.square(tf.sparse_add(x_original , negative_val))
#         loss = tf.reduce_mean(square_diff, name='loss')
#         tf.summary.scalar('loss', loss)
#     with tf.variable_scope('Optimizer'):
#         optimizer = tf.train.AdamOptimizer(\
#             learning_rate=learning_rate, 
#             name='Adam-op').minimize(loss)

In [ ]:
out_values = tf.placeholder(tf.float32, name='out_values')

In [ ]:
coo_matrix = data.tocoo()
tf_coo_matrix = tf.SparseTensorValue(
    indices=np.array([coo_matrix.row, coo_matrix.col], dtype=np.int).T,
    values=coo_matrix.data,
    dense_shape=coo_matrix.shape)

trainData = tf.SparseTensor.from_value(tf_coo_matrix)

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

out_values = fc_layer(trainData, num_features, h1, 'TestMatMult', use_relu=False)


In [ ]:
sess.run(out_values)

In [ ]:
# Define the loss function, optimizer, and accuracy for tensor (dense tensor)
# with tf.variable_scope('Train'):
#     with tf.variable_scope('Loss'):
#         loss = tf.reduce_mean(tf.losses.mean_squared_error(x_original, out), name='loss')
#         tf.summary.scalar('loss', loss)
#     with tf.variable_scope('Optimizer'):
#         optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)

# # Initializing the variables
# init = tf.global_variables_initializer()

In [ ]:
# return  SparseTensor
class NextBatchTrainData:
    '''
    slice small number of data points from the large csr_matrix
    make a SparseTensor and return it 
    '''
    def __init__(self, batch_size):
        np.random.seed(10)
        self.idx = np.arange(0 , data.shape[0])
        np.random.shuffle(self.idx)
        self.idx = self.idx[:batch_size]
        self.current = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current > len(self.idx) -1:
            raise StopIteration
        else:
            self.current += 1
            coo_matrix = data[self.current].tocoo()
            tf_coo_matrix = tf.SparseTensorValue(
                indices=np.array([coo_matrix.row, coo_matrix.col]).T,
                values=coo_matrix.data,
                dense_shape=coo_matrix.shape)

            return tf.SparseTensor.from_value(tf_coo_matrix)

In [ ]:
import pdb

In [ ]:
# # return SparseTensorValue
# class NextBatchTrainData:
#     '''
#     slice small number of data points from the large csr_matrix
#     make a SparseTensorValue and return it 
#     '''
#     def __init__(self, batch_size):
#         pdb.set_trace()
#         np.random.seed(10)
#         self.idx = np.arange(0 , data.shape[0])
#         np.random.shuffle(self.idx)
#         self.idx = self.idx[:batch_size]
#         self.current = 0
        
#     def __iter__(self):
#         return self
    
#     def __next__(self):
#         if self.current > len(self.idx) -1:
#             raise StopIteration
#         else:
#             self.current += 1
#             coo_matrix = data[self.idx].tocoo()
#             return tf.SparseTensorValue(
#                 indices=np.array([coo_matrix.row, coo_matrix.col]).T,
#                 values=coo_matrix.data,
#                 dense_shape=coo_matrix.shape)

In [ ]:
coo_matrix = data[:100].tocoo()
TrainData = tf.SparseTensorValue(
    indices=np.array([coo_matrix.row, coo_matrix.col]).T,
    values=coo_matrix.data,
    dense_shape=coo_matrix.shape)

In [ ]:
coo_matrix = test[:100].tocoo()
TestData = tf.SparseTensorValue(
    indices=np.array([coo_matrix.row, coo_matrix.col]).T,
    values=coo_matrix.data,
    dense_shape=coo_matrix.shape)

In [ ]:
%%time
C = NextBatchTrainData(5)

In [ ]:
type(TrainData)

In [ ]:
TrainData.dense_shape

In [ ]:
for i in TrainData:
    print(i)
    break

In [ ]:
TrainData.values[:10]

In [ ]:
data.data[:10]

In [ ]:
data.indices[:10]

In [ ]:
TrainData.indices[:10]

In [ ]:
data.shape

In [ ]:
denseTrain = tf.sparse_tensor_to_dense(TrainData)

#### prepare the test tensor

In [ ]:
%%time
class NextBatchTestData:
    def __init__(self, batch_size):
        np.random.seed(10)
        self.idx = np.arange(0 , test.shape[0])
        np.random.shuffle(self.idx)
        self.idx = self.idx[:batch_size]
        self.current = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current > len(self.idx) -1 :
            raise StopIteration
        else:
            self.current += 1
            test_coo_matrix = test[self.current].tocoo()
            test_tf_coo_matrix = tf.SparseTensorValue(
                indices=np.array([test_coo_matrix.row, test_coo_matrix.col]).T,
                values=test_coo_matrix.data,
                dense_shape=test_coo_matrix.shape)

            return tf.SparseTensor.from_value(test_tf_coo_matrix)

##### The out_values needs to support iteration

In [ ]:
for i in NextBatchTestData(2):
    print(i.get_shape())
#     pass

In [ ]:
test_sp_tensor.get_shape()

In [ ]:
type(test_sp_tensor)

In [ ]:
# config = tf.ConfigProto()
# config.gpu_options.allocator_type ='BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.90

In [ ]:
data.shape[0]

In [ ]:
# debug the fetch of data to sparse_placeholders
sess = tf.InteractiveSession() # using InteractiveSession instead of Session to test network in separate cell
# sess = tf.Session() 
init = tf.global_variables_initializer()
sess.run(init)
train_writer = tf.summary.FileWriter(logs_path, sess.graph)
batch_x = NextBatchTrainData(batch_size)
sess.run([NextBatchTrainData(batch_size)], feed_dict={x_original: batch_x
                                                     })

In [ ]:
# Launch the graph (session)
sess = tf.InteractiveSession() # using InteractiveSession instead of Session to test network in separate cell
# sess = tf.Session() 
init = tf.global_variables_initializer()
sess.run(init)
train_writer = tf.summary.FileWriter(logs_path, sess.graph)
num_tr_iter = int(data.shape[0] / batch_size)
global_step = 0
for epoch in range(epochs):
    print('Training epoch: {}'.format(epoch + 1))
    for iteration in range(num_tr_iter):
        batch_x = NextBatchTrainData(batch_size)
        batch_out = NextBatchTestData(batch_size)
#         print(batch_x.get_shape(), flush=True)
#         batch_x_noisy = batch_x + noise_level * np.random.normal(loc=0.0, scale=1.0, size=batch_x.shape)

        global_step += 1
        print(global_step, flush=True)
        # Run optimization op (backprop)
#         feed_dict_batch = {x_original: batch_x, x_noisy: batch_x_noisy}
        feed_dict_batch = {x_original: denseTrain, out_values: TestData} #problem 1
        _, summary_tr = sess.run([optimizer, loss], feed_dict=feed_dict_batch)
#         feed_dict_batch = {x_original: data[:batch_size]} # just run the optimizer
#         opti_result = sess.run([optimizer], feed_dict=feed_dict_batch)
        sess.run(optimizer)
        
        train_writer.add_summary(summary_tr, global_step)

        if iteration % display_freq == 0:
            # Calculate and display the batch loss and accuracy
            loss_batch = sess.run(loss,
                                  feed_dict=feed_dict_batch)
            print("iter {0:3d}:\t Reconstruction loss={1:.3f}".
                  format(iteration, loss_batch))

    # Run validation after every epoch
#     x_valid_original  = mnist.validation.images
    x_valid_original  = test_sp_tensor
#     x_valid_noisy = x_valid_original + noise_level * np.random.normal(loc=0.0, scale=1.0, size=x_valid_original.shape)

    feed_dict_valid = {x_original: x_valid_original}
    loss_valid = sess.run(loss, feed_dict=feed_dict_valid)
    print('---------------------------------------------------------')
    print("Epoch: {0}, validation loss: {1:.3f}".
          format(epoch + 1, loss_valid))
    print('---------------------------------------------------------')
 

In [ ]:
sess.close()